In [166]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import numpy as np
import sklearn
from datetime import datetime
import matplotlib.pyplot as plt

In [167]:
data = pd.read_csv("rfm_data.csv")

In [168]:
data.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris


In [169]:
data.dtypes

CustomerID              int64
PurchaseDate           object
TransactionAmount     float64
ProductInformation     object
OrderID                 int64
Location               object
dtype: object

In [170]:
datetime.now()

datetime.datetime(2023, 7, 10, 6, 14, 8, 883865)

In [171]:
data['PurchaseDate'] = pd.to_datetime(data['PurchaseDate'])

In [172]:
# calculating recency by calculating days past since last purchase from today
data['Recency'] = data['PurchaseDate'].apply(lambda x: (datetime.now().date() - x.date()).days)

In [173]:
data.groupby('CustomerID')['OrderID'].count()

CustomerID
1011    2
1025    1
1029    1
1046    1
1049    1
       ..
9941    1
9950    1
9954    1
9985    1
9991    1
Name: OrderID, Length: 946, dtype: int64

In [174]:
# Calculate Frequency
frequency_data = data.groupby('CustomerID')['OrderID'].count().reset_index()
frequency_data.rename(columns={'OrderID': 'Frequency'}, inplace=True)
data = data.merge(frequency_data, on='CustomerID', how='left')

In [175]:
# Calculate Monetary Value
monetary_data = data.groupby('CustomerID')['TransactionAmount'].sum().reset_index()
monetary_data.rename(columns={'TransactionAmount': 'MonetaryValue'}, inplace=True)
data = data.merge(monetary_data, on='CustomerID', how='left')

In [176]:
frequency_data

,CustomerID,Frequency
0,1011,2
1,1025,1
2,1029,1
3,1046,1
4,1049,1
...,...,...
941,9941,1
942,9950,1
943,9954,1
944,9985,1


In [177]:
data.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,90,1,943.31
1,2188,2023-04-11,463.70,Product A,176819,London,90,1,463.70
2,4608,2023-04-11,80.28,Product A,340062,New York,90,1,80.28
3,2559,2023-04-11,221.29,Product A,239145,London,90,1,221.29
4,9482,2023-04-11,739.56,Product A,194545,Paris,90,1,739.56


In [178]:
# Define scoring criteria for each RFM value
recency_scores = [5, 4, 3, 2, 1]  # Higher score for lower recency (more recent)
frequency_scores = [1, 2, 3, 4, 5]  # Higher score for higher frequency
monetary_scores = [1, 2, 3, 4, 5]  # Higher score for higher monetary value

# Calculate RFM scores
data['RecencyScore'] = pd.cut(data['Recency'], bins=5, labels=recency_scores)
data['FrequencyScore'] = pd.cut(data['Frequency'], bins=5, labels=frequency_scores)
data['MonetaryScore'] = pd.cut(data['MonetaryValue'], bins=5, labels=monetary_scores)

In [179]:
# Convert RFM scores to numeric type
data['RecencyScore'] = data['RecencyScore'].astype(int)
data['FrequencyScore'] = data['FrequencyScore'].astype(int)
data['MonetaryScore'] = data['MonetaryScore'].astype(int)

In [180]:
# Calculate RFM score by combining the individual scores
data['RFM_Score'] = data['RecencyScore'] + data['FrequencyScore'] + data['MonetaryScore']

# Create RFM segments based on the RFM score
segment_labels = ['Low-Value', 'Mid-Value', 'High-Value']
data['Value Segment'] = pd.qcut(data['RFM_Score'], q=3, labels=segment_labels)

In [181]:
score_counts = data['RFM_Score'].value_counts()

In [182]:
score_counts

RFM_Score
7     196
6     190
5     180
4     173
8     117
3      82
9      28
11     15
10     12
12      4
13      2
15      1
Name: count, dtype: int64

In [183]:
# Create a bar plot using Plotly
fig = go.Figure(
    data=[go.Bar(x=score_counts.index, y=score_counts.values)]
)

# Add labels and title
fig.update_layout(
    xaxis_title='RFM Score',
    yaxis_title='Count',
    title='RFM Score Distribution'
)
fig.update_xaxes(type='category', categoryorder='total descending')

# Display the plot
fig.show()

In [184]:
# RFM Segment Distribution
segment_counts = data['Value Segment'].value_counts().reset_index()
segment_counts.columns = ['Value Segment', 'Count']

pastel_colors = px.colors.qualitative.Pastel

# Create the bar chart
fig_segment_dist = px.bar(segment_counts, x='Value Segment', y='Count',
                          color='Value Segment', color_discrete_sequence=pastel_colors,
                          title='RFM Value Segment Distribution')

# Update the layout
fig_segment_dist.update_layout(xaxis_title='RFM Value Segment',
                               yaxis_title='Count',
                               showlegend=False)

# Show the figure
fig_segment_dist.show()

In [185]:
# Create a new column for RFM Customer Segments
data['RFM Customer Segments'] = ''

# Assign RFM segments based on the RFM score
data.loc[data['RFM_Score'] >= 9, 'RFM Customer Segments'] = 'Champions'
data.loc[(data['RFM_Score'] >= 6) & (data['RFM_Score'] < 9), 'RFM Customer Segments'] = 'Potential Loyalists'
data.loc[(data['RFM_Score'] >= 5) & (data['RFM_Score'] < 6), 'RFM Customer Segments'] = 'At Risk Customers'
data.loc[(data['RFM_Score'] >= 4) & (data['RFM_Score'] < 5), 'RFM Customer Segments'] = "Can't Lose"
data.loc[(data['RFM_Score'] >= 3) & (data['RFM_Score'] < 4), 'RFM Customer Segments'] = "Lost"

# Print the updated data with RFM segments
print(data[['CustomerID', 'RFM Customer Segments']])

     CustomerID RFM Customer Segments
0          8814            Can't Lose
1          2188                  Lost
2          4608                  Lost
3          2559                  Lost
4          9482            Can't Lose
..          ...                   ...
995        2970   Potential Loyalists
996        6669   Potential Loyalists
997        8836   Potential Loyalists
998        1440   Potential Loyalists
999        4759   Potential Loyalists

[1000 rows x 2 columns]


In [186]:
segment_product_counts = data.groupby(['Value Segment', 'RFM Customer Segments']).size().reset_index(name='Count')

segment_product_counts = segment_product_counts.sort_values('Count', ascending=False)

fig_treemap_segment_product = px.treemap(segment_product_counts,
                                         path=['Value Segment', 'RFM Customer Segments'],
                                         values='Count',
                                         color='Value Segment', color_discrete_sequence=px.colors.qualitative.Pastel,
                                         title='RFM Customer Segments by Value')
fig_treemap_segment_product.show()

In [187]:
# use backticks when using .query on names containing space
data.query("`RFM Customer Segments` == 'Champions'")

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score,Value Segment,RFM Customer Segments
63,5557,2023-04-14,511.86,Product D,522866,London,87,3,2379.45,1,5,5,11,High-Value,Champions
180,5557,2023-04-22,996.98,Product D,765620,Tokyo,79,3,2379.45,1,5,5,11,High-Value,Champions
220,3938,2023-04-25,806.29,Product A,520400,New York,76,2,1731.53,2,3,4,9,High-Value,Champions
235,4804,2023-04-25,690.50,Product C,183336,Tokyo,76,3,2073.33,2,5,5,12,High-Value,Champions
249,7363,2023-04-26,262.45,Product C,778120,New York,75,3,1386.32,2,5,3,10,High-Value,Champions
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951,1480,2023-06-07,812.58,Product C,161437,Tokyo,33,2,1621.62,5,3,4,12,High-Value,Champions
952,9066,2023-06-07,985.51,Product C,614646,London,33,1,985.51,5,1,3,9,High-Value,Champions
965,1803,2023-06-08,610.34,Product D,973294,Paris,32,2,1033.13,5,3,3,11,High-Value,Champions
972,7296,2023-06-09,984.60,Product C,846172,New York,31,1,984.60,5,1,3,9,High-Value,Champions


In [188]:
# Filter the data to include only the customers in the Champions segment
champions_segment = data[data['RFM Customer Segments'] == 'Champions']

fig = go.Figure()
fig.add_trace(go.Box(y=champions_segment['RecencyScore'], name='Recency'))
fig.add_trace(go.Box(y=champions_segment['FrequencyScore'], name='Frequency'))
fig.add_trace(go.Box(y=champions_segment['MonetaryScore'], name='Monetary'))

fig.update_layout(title='Distribution of RFM Values within Champions Segment',
                  yaxis_title='RFM Value',
                  showlegend=True)

fig.show()

In [189]:
correlation_matrix = champions_segment[['RecencyScore', 'FrequencyScore', 'MonetaryScore']].corr()

# Visualize the correlation matrix using a heatmap
fig_heatmap = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale='RdBu',
    colorbar=dict(title='Correlation')))

fig_heatmap.update_layout(title='Correlation Matrix of RFM Values within Champions Segment')

fig_heatmap.show()

In [190]:
data['RFM Customer Segments'].value_counts()

RFM Customer Segments
Potential Loyalists    503
At Risk Customers      180
Can't Lose             173
Lost                    82
Champions               62
Name: count, dtype: int64

In [191]:
import plotly.colors

pastel_colors = plotly.colors.qualitative.Pastel

segment_counts = data['RFM Customer Segments'].value_counts()

# Create a bar chart to compare segment counts
fig = go.Figure(data=[go.Bar(x=segment_counts.index, y=segment_counts.values,
                             marker=dict(color=pastel_colors))])

# Set the color of the Champions segment as a different color
# champions_color = 'rgb(158, 202, 225)'
# fig.update_traces(marker_color=[champions_color if segment == 'Champions' else pastel_colors[i]
#                                 for i, segment in enumerate(segment_counts.index)],
#                   marker_line_color='rgb(8, 48, 107)',
#                   marker_line_width=1.5, opacity=0.6)

# Update the layout
fig.update_layout(title='Comparison of RFM Segments',
                  xaxis_title='RFM Segments',
                  yaxis_title='Number of Customers',
                  showlegend=False)

fig.show()

In [193]:
# Calculate the average Recency, Frequency, and Monetary scores for each segment
segment_scores = data.groupby('RFM Customer Segments')[
    ['RecencyScore', 'FrequencyScore', 'MonetaryScore']].mean().reset_index()
# Create a grouped bar chart to compare segment scores
fig = go.Figure()
# Add bars for Recency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['RecencyScore'],
    name='Recency Score',
    # marker_color='rgb(158,202,225)'
))
# Add bars for Frequency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['FrequencyScore'],
    name='Frequency Score',
    # marker_color='rgb(94,158,217)'
))
# Add bars for Monetary score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['MonetaryScore'],
    name='Monetary Score',
    # marker_color='rgb(32,102,148)'
))
# Update the layout
fig.update_layout(
    title='Comparison of RFM Segments based on Recency, Frequency, and Monetary Scores',
    xaxis_title='RFM Segments',
    yaxis_title='Score',
    barmode='group',
    showlegend=True
)
fig.show()